In [1]:
# отключаем предупреждения Anaconda
import warnings
warnings.simplefilter('ignore')

# импортируем библиотеку pandas
import pandas as pd

# cчитываем данные и выводим первые 5 наблюдений
gl = pd.read_csv('Notebooks/Data/game_logs.csv')
gl.head()

,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,...,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info
0,18710504,0,Thu,CL1,na,1,FW1,na,1,0,...,Ed Mincher,7.0,mcdej101,James McDermott,8.0,kellb105,Bill Kelly,9.0,NaN,Y
1,18710505,0,Fri,BS1,na,1,WS3,na,1,20,...,Asa Brainard,1.0,burrh101,Henry Burroughs,9.0,berth101,Henry Berthrong,8.0,HTBF,Y
2,18710506,0,Sat,CL1,na,2,RC1,na,1,12,...,Pony Sager,6.0,birdg101,George Bird,7.0,stirg101,Gat Stires,9.0,NaN,Y
3,18710508,0,Mon,CL1,na,3,CH1,na,1,12,...,Ed Duffy,6.0,pinke101,Ed Pinkham,5.0,zettg101,George Zettlein,1.0,NaN,Y
4,18710509,0,Tue,BS1,na,2,TRO,na,1,9,...,Steve Bellan,5.0,pikel101,Lip Pike,3.0,cravb101,Bill Craver,6.0,HTBF,Y


Мы можем воспользоваться методом **.info()**, чтобы получить основные сведения о нашем
датафрейме, включая его размер, информацию о типах данных и использовании памяти.

По умолчанию pandas вычисляет лишь приблизительный объем памяти, используемый
датафреймом, в целях экономии времени. Поскольку нас интересует
точность, мы установим параметру **memory_usage значение 'deep'**, чтобы получить
точную информацию.

In [2]:
# выводим точную информацию об использовании памяти
gl.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171907 entries, 0 to 171906
Columns: 161 entries, date to acquisition_info
dtypes: float64(77), int64(6), object(78)
memory usage: 859.4 MB


In [3]:
# посмотрим, сколько памяти в среднем используют столбцы определенного типа
for dtype in ['float', 'int', 'object']:
    selected_dtype = gl.select_dtypes(include=[dtype])
    mean_usage_b = selected_dtype.memory_usage(deep=True).mean()
    mean_usage_mb = mean_usage_b / 1024 ** 2
    print(f'Использование памяти в среднем для столбцов: {dtype}: {mean_usage_mb} MB')

Использование памяти в среднем для столбцов: float: 1.294733194204477 MB
Использование памяти в среднем для столбцов: int: 1.1242000034877233 MB
Использование памяти в среднем для столбцов: object: 9.500870656363572 MB


#### Подтипы
Многие типы в библиотеке pandas имеют несколько подтипов, которые могут
использовать меньшее количество байтов для представления каждого значения.
Например, тип float имеет подтипы float16, float32 и float64. Число, следующее
после названия типа, указывает количество битов, которое используется данным
типом для представления значений. Например, подтипы, которые мы только что
указали, используют соответственно 2, 4, 8 и 16 байтов.

Мы можем воспользоваться классом **numpy.iinfo**, чтобы посмотреть минималь-
ное и максимальное значения для каждого целочисленного подтипа.

In [4]:
# импортируем библиотеку numpy
import numpy as np

# посмотрим минимальное и максимальное значения для каждого целочисленного типа
int_types = ['uint8', 'int8', 'int16']
for it in int_types:
    print(np.iinfo(it))

Machine parameters for uint8
---------------------------------------------------------------
min = 0
max = 255
---------------------------------------------------------------

Machine parameters for int8
---------------------------------------------------------------
min = -128
max = 127
---------------------------------------------------------------

Machine parameters for int16
---------------------------------------------------------------
min = -32768
max = 32767
---------------------------------------------------------------



#### Оптимизация числовых столбцов с помощью понижающего преобразования
Мы можем использовать функцию <mark>pd.to_numeric()</mark>, чтобы выполнить понижающее
преобразование наших числовых типов. Мы воспользуемся DataFrame.select_
dtypes для выбора только целочисленных столбцов, тогда мы будем оптимизировать
типы и сравнивать использование памяти.

In [5]:
# мы будем довольно часто подсчитывать использование памяти,
# поэтому напишем функцию, которая сэкономит нам немного времени
def mem_usage(pandas_obj):
    if isinstance(pandas_obj, pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else:  # предположим, что если это не датафрейм, то серия
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # преобразуем байты в мегабайты
    return f'{usage_mb} mb'

In [25]:
# выполняем понижающее преобразование для столбцов типа int
gl_int = gl.select_dtypes(include=['int'])
converted_int = gl_int.apply(pd.to_numeric, downcast='unsigned')

print(mem_usage(gl_int))
print(mem_usage(converted_int))

compare_ints = pd.concat([gl_int.dtypes,
                          converted_int.dtypes], axis='columns')
compare_ints.columns = ['before', 'after']
compare_ints.apply(pd.Series.value_counts)

7.8694000244140625 mb
1.475611686706543 mb


,before,after
uint8,NaN,5.0
uint32,NaN,1.0
int64,6.0,NaN


In [27]:
# выполняем понижающее преобразование для столбцов типа float
gl_float = gl.select_dtypes(include=['float'])
converted_float = gl_float.apply(pd.to_numeric, downcast='float')

print(mem_usage(gl_float))
print(mem_usage(converted_float))

compare_floats = pd.concat([gl_float.dtypes, converted_float.dtypes], axis='columns')
compare_floats.columns = ['before', 'after']
compare_floats.apply(pd.Series.value_counts)

100.98918914794922 mb
50.49465560913086 mb


,before,after
float32,NaN,77.0
float64,77.0,NaN


Давайте создадим копию нашего исходного датафрейма, заменив исходные числовые столбцы оптимизированными числовыми столбцами, и посмотрим, каким будет общее использование памяти.

In [ ]:
# создаем копию исходного датафрейма
optimized_gl = gl.copy()

# заменяем исходные числовые столбцы оптимизированными
optimized_gl[converted_int.columns] = converted_int
optimized_gl[converted_float.columns] = converted_float

# смотрим использование памяти
print(mem_usage(gl))
print(mem_usage(optimized_gl))

Хотя мы значительно сократили использование памяти нашими числовыми
столбцами, в целом же мы уменьшили потребление памяти нашим датафреймом
только на **7 %**. В значительной мере снижение использования памяти будет зави-
сеть от оптимизации столбцов типа object.

#### Сравнение способов хранения числовых и строковых значений